#  Вложенные функции, замыкания и декораторы

## 1. Функции

**Функция** — часть программы, которую можно вызвать из другого места программы.

Все в Python объекты. И даже функции. Это значит, что у функций есть
- атрибуты
- и методы.

От остальных объектов функции отличаются тем, что их можно вызвать*. Объекты, которые можно вызвать, называют `Callable`-объектами. У них есть метод `__call__()`.

\* С точки зрения синтаксиса еще можно вызывать классы

### Как определить функцию

In [36]:
# Функция определяется таким синтаксисом
def plus_one(x: int) -> int:
    """Функция возвращает увеличенное на 1 целое число"""
    return x+1

Это избыточное определение. Из избыточного здесь использованы:
- строка документирования — `docsting`,
- и анотация функции.

На самом деле можно описать эту же функцию компактней. 

In [51]:
# Функция plus_one без анотаций и документации
def plus_one_simple(x): return x+1

### Функция как объект

Как у любого объекта в python, у функции есть:
- идентификатор,
- тип.

In [47]:
# У функции plus_one эти параметры выглядят так
id(plus_one), type(plus_one)

(4510970608, function)

В CPython идентификатор — **адрес объекта** в виртуальной памяти

In [48]:
# Идентификатор в шестнадцатиричном формате — адрес функции plus_one
hex(id(plus_one))

'0x10cdff2f0'

Все атрибуты и методы функции как объекта можно посмотреть:

In [45]:
import inspect
list(filter(lambda x: x[0] != "__globals__", sorted(inspect.getmembers(plus_one))))
# Здесь мы выбросили поле "__globals__", чтобы не засорять вывод

[('__annotations__', {'x': int, 'return': int}),
 ('__call__', <method-wrapper '__call__' of function object at 0x10cdff2f0>),
 ('__class__', function),
 ('__closure__', None),
 ('__code__',
  <code object plus_one at 0x10d59f780, file "<ipython-input-36-59ce0af9670a>", line 2>),
 ('__defaults__', None),
 ('__delattr__',
  <method-wrapper '__delattr__' of function object at 0x10cdff2f0>),
 ('__dict__', {}),
 ('__dir__', <function function.__dir__()>),
 ('__doc__', 'Функция возвращает увеличенное на 1 целое число'),
 ('__eq__', <method-wrapper '__eq__' of function object at 0x10cdff2f0>),
 ('__format__', <function function.__format__(format_spec, /)>),
 ('__ge__', <method-wrapper '__ge__' of function object at 0x10cdff2f0>),
 ('__get__', <method-wrapper '__get__' of function object at 0x10cdff2f0>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of function object at 0x10cdff2f0>),
 ('__gt__', <method-wrapper '__gt__' of function object at 0x10cdff2f0>),
 ('__hash__', <metho

### Как вызвать функцию

In [39]:
#  Вызов функции, ожидаем ответ 2
plus_one(1)

2

In [40]:
#  Можно явно вызвать метод call, ожидаем ответ 2
plus_one.__call__(1)

2

### Как функции устроены

In [41]:
# Байт-код функции function_name
plus_one.__code__.co_code

b'|\x00d\x01\x17\x00S\x00'

In [42]:
# Дизассемблированное тело функции function_name
import dis
dis.dis(plus_one)

  4           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 RETURN_VALUE


Если заглянуть во внутренности интерпретатора (CPython), то функция описывается следующей струтурой: https://github.com/python/cpython/blob/3.7/Include/funcobject.h

### Почитать
1. [The Python Language Reference. Data model](https://docs.python.org/3/reference/datamodel.html#objects-values-and-types)
2. [The Python Language Reference. Inspect live objects](https://docs.python.org/3/library/inspect.html)
3. [PEP 3107 -- Function Annotations](https://www.python.org/dev/peps/pep-3107/)
4. [PEP 257 -- Docstring Conventions](https://www.python.org/dev/peps/pep-0257/)

## 2. Вложенные функции

**Вложенная функция** — функция, которая определена внутри другой функции.

При работе с вложенными функциями надо учитывать области видимости.

### Область видимости в Python — LEGB

В Python есть 4 области видимости. Расположены они как показано на рисунке.

![title](img/LEGB.png)

Стрелки на рисунке показывают в какой последовательности Python обходит области видимости. Следующий код показывает как распределены области относительно вложенной функции inner.

In [64]:
# (built-in) — область системных имен

# global — область модуля
def outer():
    # enclosed — область функции-обёртки 
    def inner():
        # local — область внутри функции
        pass

### Зачем нужны вложенные функции?

Зачем это может быть нужно? Можно выделить 3 примера:
1. чтобы скрыть функцию в глобальной области видимости,
2. чтобы вынести «лишний» код из функцию в обёртку,
3. чтобы реализовать замыкания (см. следующий раздел).

#### Пример 1. Чтобы скрыть функцию — инкапсуляция

In [53]:
# Вложенная функция inner внутри plus_one_outer
def plus_one_outer(x: int) -> int:
    """Функция возвращает увеличенное на 1 целое число"""
    def inner(y: int) -> int: return y+1
    return inner(x)
    

In [55]:
#  Вызов функции, ожидаем ответ 2
plus_one_outer(1)

2

In [57]:
# Вложенная функция недоступна (должна быть ошибка)
inner(1)

NameError: name 'inner' is not defined

Вложенные функции дают накладные расходы

In [56]:
import dis
dis.dis(plus_one_outer)

  4           0 LOAD_GLOBAL              0 (int)
              2 LOAD_GLOBAL              0 (int)
              4 LOAD_CONST               1 (('y', 'return'))
              6 BUILD_CONST_KEY_MAP      2
              8 LOAD_CONST               2 (<code object inner at 0x10d59fc00, file "<ipython-input-53-07493295d00f>", line 4>)
             10 LOAD_CONST               3 ('plus_one_outer.<locals>.inner')
             12 MAKE_FUNCTION            4
             14 STORE_FAST               1 (inner)

  5          16 LOAD_FAST                1 (inner)
             18 LOAD_FAST                0 (x)
             20 CALL_FUNCTION            1
             22 RETURN_VALUE

Disassembly of <code object inner at 0x10d59fc00, file "<ipython-input-53-07493295d00f>", line 4>:
  4           0 LOAD_FAST                0 (y)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 RETURN_VALUE


#### Пример 2. Чтобы вынести «лишний» код из функции в обёртку

In [59]:
def factorial(x: int) -> int:
    """Функция вычисляет факториал целого числа"""
    def calc_factorial(y: int) -> int: return y * calc_factorial(y-1) if y!=0 else 1
    if x<0:
        return -1
    return calc_factorial(x)
    

In [62]:
factorial(4)

24

## 3. Замыкания

**Замыкание** — вложенная функция, которая запоминает значения окружения, с которым она была вызвана. Говорят, что функция «замыкается» на значения переменных окружения. По сути это техника параметризированной генерации функций.

Рассмотрим простой пример замыкания

In [83]:
from typing import Callable

# Функция-обёртка принимает возвращает внутренную функцию, которая «замкнута» на значение a
def gen_mul(a: int) -> Callable[[int], int]:
    def inner(b: int) -> int:
        return a*b
    return inner

In [84]:
# gen_mul возвращает функцию, которая будет всегда умножать на 2
double = generate_mul(2)

In [85]:
# Проверим (должно быть 6)
double(3)

6

In [86]:
# Можно возвращаемую функцию не сохранять
gen_mul(2)(3)

6

Функции, которые возвращают другие функции, называются **«фабриками функций»**.

## 4. Декораторы

**Декоратор** — «синтаксический сахар» для функции-обёртки вокруг другой функции. Обычно декоратор используют, чтобы добавить новое поведение другой функции без изменения ее тела.

In [93]:
# Возьмем простую функцию возведения в квадрат
def sqr(x: int) -> int: return x*x
sqr(5)

25

### Пример элементарного декоратора

Мы хотим обёрнуть функцию возведения в квадрат другой функцией, чтобы добавить новые возможности.
Ниже приведен пример фабрики функций, которая возвращает функцию-обёртку. Эта функция-обёртка выполняет новый код
и вызывает оборачивемую функцию.

In [185]:
from time import perf_counter_ns
from typing import Callable

# Фабрика функций, которая генерирует обернутые функции func для отладки вызова и результата
def make_debugable(func: Callable[[int], int]) -> Callable[[int], int]:
    def wrapper(x: int) -> int:
        print(f"[DEBUG] Launch function {func} with x={x}")
        start_time_ns = perf_counter_ns()
        result = func(x)
        stop_time_ns = perf_counter_ns()
        duration_ns = stop_time_ns-start_time_ns
        print(f"[DEBUG] Time: {duration_ns}ns")
        return result
    return wrapper

Обернём функцию sqr, сгенерированное значение будем хранить в sqrt2.

In [186]:
sqrt2 = make_debugable(sqr)
sqrt2(5)

[DEBUG] Launch function <function sqr at 0x10cefc158> with x=5
[DEBUG] Time: 1441ns


25

Синтаксический сахар декораторов позволяет описать такое поведение короче. При этом декоратор генерирует обёрнутую функцию с таким же именем как у обораичиваемой.

In [178]:
# Следующий код эквивалентен: sqr3 = make_debugable(sqr3)
@make_debugable
def sqr3(x: int) -> int: return x*x

sqr3(4)

[DEBUG] Launch function <function sqr3 at 0x10d2ce730> with x=4
[DEBUG] Time: 1208ns


16

#### Практичный пример применения декоратора `make_debugable`

Сравним скорость работы встроенной функции `sum` с написанной «руками» с помощью декоратора `make_debugable`.

In [179]:
@make_debugable
def sum_1(n: int) -> int:
    """Суммирование числе от 1 до n в цикле for"""
    s = 0
    for i in range(n):
        s += i
    return s

In [180]:
@make_debugable
def sum_2(n: int) -> int:
    """Суммирование числе от 1 до n встроенной функцией"""
    return sum(range(n))

In [181]:
sum_1(10000)

[DEBUG] Launch function <function sum_1 at 0x10d2ce158> with x=10000
[DEBUG] Time: 590115ns


49995000

In [182]:
sum_2(10000)

[DEBUG] Launch function <function sum_2 at 0x10cdefc80> with x=10000
[DEBUG] Time: 214377ns


49995000

Наглядно видно, что встроенная функция производительней.

### Пример рабочего декоратора

По сути декоратор возвращает другую функцию. Если проверить документацию к функции:

In [168]:
# Проверка справки (не должен вернуть строку)
help(sum_1)

Help on function wrapper in module __main__:

wrapper(x: int) -> int



Чтобы возвращаемая функция была похожа на оборачиваемую надо скопировать внутреннии атрибуты.

In [184]:
from functools import wraps
from time import perf_counter_ns
from typing import Callable

# Улучшенная фабрика функций, которая генерирует обернутые функции func для отладки вызова и результата
def make_debugable_real(func: Callable[[int], int]) -> Callable[[int], int]:
    @wraps(func) # Декоратор из библиотеки для копирования внутренних атрибутов
    def wrapper(x: int) -> int:
        print(f"[DEBUG] Launch function {func} with x={x}")
        start_time_ns = perf_counter_ns()
        result = func(x)
        stop_time_ns = perf_counter_ns()
        duration_ns = stop_time_ns-start_time_ns
        print(f"[DEBUG] Time: {duration_ns}ns")
        return result
    return wrapper

In [173]:
@make_debugable_real
def sum_3(n: int) -> int:
    """Суммирование числе от 1 до n встроенной функцией"""
    return sum(range(n))

In [183]:
# Проверка справки (теперь должен вернуть строку)
help(sum_3)

Help on function sum_3 in module __main__:

sum_3(n: int) -> int
    Суммирование числе от 1 до n встроенной функцией



### Пример рабочего декоратора с параметрами

Чтобы декоратор принимал аргументы, надо сформировать замыкание фабрики обёрток с параметром декоратора.

In [217]:
from functools import wraps
from typing import Callable

# Фабарика генераторов функций, которая позволяет использовать параметры, функция умножает результат функции на число.
def mul(p: int) -> Callable[[x],  Callable[[], int]]:
    def decorator(func: Callable[[], int]) -> Callable[[], int]:
        @wraps(func)
        def wrapper(*args) -> int: # Упаковали параметры (см. ниже)
            return p*func(*args) # Распаковали параметры обратно (см. ниже)
        return wrapper
    return decorator

Объявим функцию с двумя декораторами. Декораторы применяются последовательнос снизу вверх.

In [218]:
@mul(2)
@mul(4)
def f(x: int, y: int) -> int:
    return x+y

In [219]:
# Должно быть 24 так как 2(4(1+2)) = 24
f(1,2)

24

#### Упаковка и распаковка параметров

При работе с последовательностями можно собирать значения в переменные. Это называется **упаковка**. Синтаксис такой.

In [214]:
a, *b, c = [2, 7, 5, 6, 3, 4, 1]

В `b` теперь список всего того, что не попало в `a` и `c`

In [215]:
print(b)

[7, 5, 6, 3, 4]


Эти значения можно подставить используя **распаковку**.

In [220]:
print(*b)

7 5 6 3 4


Результат вывода разный. В первом случае вызов эквивалентен `print([b1, b2, b3])`, во втором — `print(b1, b2, b3)`.

**NB!** Упаковка и распоковка доступна и для словарей. Для этого используется **.